In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost import plot_tree

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import train_test_split

import seaborn as sns
import missingno as msno

In [ ]:
## read a file 
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

## Exploratory Data Analysis (EDA)

Let's take a look all size of each file 

In [ ]:
# which is the starting date of train and final date of train?
print('starting date of train dataset:' +train['date'].iloc[0])
print('last date of train dataset:' +train['date'].iloc[-1])
print('starting date of test dataset:' +test['date'].iloc[0])
print('last date of test dataset:' +test['date'].iloc[-1])
#total number of product family 
print('\n\nTotal number of product family:',len(train['family'].unique()))
#total number of the store in this dataset
print('Total number of stores:',len(stores['store_nbr'].unique()))

In [ ]:
train.dtypes

In [ ]:
oil.isnull().sum()

In [ ]:
oil.head()

In [ ]:
oil.rename(columns={'dcoilwtico': 'oil_price'}, inplace=True)
oil['date'] = pd.to_datetime(oil['date'])
fig = px.line(oil,x="date", y="oil_price")
fig.show()

In [ ]:
start_date = datetime(2013,1,2)
end_date = datetime(2017,8,15)
date_list = pd.date_range(start_date, end_date, freq='D')
oil_df = pd.DataFrame({'date': date_list})
oil_df = pd.merge(oil_df, oil, on='date', how='left')

In [ ]:
## handle the missing 
oil_df['oil_price'] = oil_df['oil_price'].interpolate(method='pchip').ffill().bfill()

In [ ]:
fig = px.line(oil_df,x="date", y="oil_price")
fig.show()

In [ ]:
transactions.isnull().sum()

In [ ]:
transactions.head(3)

In [ ]:
transactions.tail(3)

In [ ]:
transactions[['Year', 'Month', 'day']] = transactions['date'].str.split('-', expand=True)
transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')
transactions.set_index('date', inplace=True)

Let's view the transaction in each month 

In [ ]:
transactions_month = transactions.groupby(['Year','Month']).agg({'transactions': 'sum'}).reset_index()
fig = px.line(transactions_month,x="Month", y="transactions",color="Year",markers=True,color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_layout(
    title="transactions in each month",
    xaxis_title="month",
    yaxis_title="number of transactions",
)
fig.show()

**insight** from the data we can that on december have the highest transactions and slowest on the february, let view in smaller time frame

In [ ]:
fig = px.line(transactions.loc['2013-01-01' : '2013-12-31'], y="transactions", color="store_nbr")
fig.update_layout(
    title="transactions 2013-01-01 to 2013-12-31",
    xaxis_title="date",
    yaxis_title="number of transactions",
)
fig.show()

- **insight** if look carefully we will see the distance between the two peak is 7 days or 1 week and the highest transaction of year is on 23 december before 2 day Christmas Day. 
- **note** you can change the range of transaction 2013 to 2014,2015,2016,2017 the result are the same

In [ ]:
transactions

Let's view the day that customer buy the grocery

In [ ]:
transactions['weekday'] = transactions.index.weekday
transactions['weekday'].replace([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)
transactions.head(10)

In [ ]:
transactions.groupby('weekday')['transactions'].mean()

In [ ]:
fig = px.bar(transactions.groupby('weekday')['transactions'].mean(), title="Average Customer Spending in each weekday")
fig.show()

In [ ]:
fig = px.bar(transactions.groupby('day')['transactions'].mean(),title="Average Customer Spending in each day month")
fig.show()

**insight** customer trend to go buy the grocery on saturaday and Sunday the second ,The other interesting point is that in 1 and 31 date. 

In [ ]:
train.head(5)

In [ ]:
train.nunique()

In [ ]:
test.nunique()

- number of store is 54 stores
- number of product family is 33 stores

view relation family product and sales 

In [ ]:
train[['Year', 'Month', 'day']] = train['date'].str.split('-', expand=True)
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')
train.set_index('date', inplace=True)
train['weekday'] = train.index.weekday
train['weekday'].replace([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)

In [ ]:
sale_family = train.groupby(['Year','Month','family']).agg({'sales': 'mean'}).reset_index()
g = sns.FacetGrid(sale_family, col="family",height=4.0,col_wrap=4,sharex=False,sharey=False)
g.map_dataframe(sns.lineplot, x="Month", y="sales",hue="Year",palette="viridis")
g.add_legend()

**insight**
- The book sale all most 0 unit sale
- Frozen food sale sky rocket in December same as the Liquor,wine,beer
- SeaFood sale is decrease in December which different from other family that the highest sale on December.
- School and Office is peak in range 8-9 month

Let's see relation oil price and sale in each family product

In [ ]:
train

In [ ]:
oil['date'] = pd.to_datetime(oil['date'], format='%Y-%m-%d')
oil.set_index('date', inplace=True)
oil

In [ ]:
train = train.join(oil, how='left')
train.drop(columns=['id'],inplace=True)
train

In [ ]:
train.dtypes

In [ ]:
family_oil = train.groupby(['family','oil_price']).agg({'sales': 'mean'}).reset_index()

In [ ]:
g = sns.FacetGrid(family_oil, col="family", height=4, col_wrap=4,sharex=False,sharey=False)
g.map_dataframe(sns.regplot, x="oil_price", y="sales",scatter_kws = {"color": "black", "alpha": 0.5},line_kws={"color": "red"})
g.add_legend()

***note:*** the relation between the oil price and sale in each family is not strong.

In [ ]:
holidays

In [ ]:
sns.countplot(x=holidays['type'],order = holidays['type'].value_counts().index)
plt.title('Count type of the hoilday')

In [ ]:
sns.countplot(x=holidays['locale'],order = holidays['locale'].value_counts().index)

In [ ]:
sns.countplot(x=holidays['transferred'],order = holidays['transferred'].value_counts().index)

Does earthquake in 16 April 2016 effect the sale in some store or not ?

## Data Preprocessing

In [ ]:
df_train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
df_train

In [ ]:
df_stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
df_stores

In [ ]:
# get the dataframe that contain one product family 
def get_one_family(df,family):
    return df[df['family'] == family]

In [ ]:
# process the train dataset
df_train[['Year', 'Month', 'day']] = df_train['date'].str.split('-', expand=True)
df_train['date'] = pd.to_datetime(df_train['date'], format='%Y-%m-%d')##
df_train['Quarter'] = df_train['date'].dt.quarter
df_train['weekday'] = df_train["date"].dt.dayofweek
#df_train['weekday'].replace([0, 1, 2, 3, 4, 5, 6], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)

## join stores.csv
df_train = pd.merge(df_train,df_stores,how='left', on='store_nbr')

## drop unnecessary stuff
df_train.drop(['id','onpromotion'], axis=1, inplace=True)
df_train

In [ ]:
#one hot encoded 
#feature_one_hot_encoded = ['weekday','day','store_nbr']
'''
feature_one_hot_encoded = ['store_nbr']
for feature in feature_one_hot_encoded:
    temp = pd.get_dummies(df_train[feature],prefix=feature)
    df_train = pd.concat([df_train,temp],axis=1)
    df_train.drop([feature], axis=1, inplace=True)
'''
df_train


In [ ]:
## process on hoilday data
df_holiday = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')

df_holiday = df_holiday[df_holiday['transferred'] == False].copy()
df_holiday['holiday_type'] = df_holiday['type']
df_holiday.drop(['transferred', 'description', 'type'], axis=1, inplace=True)

df_national_holiday = df_holiday[df_holiday['locale'] == 'National'].copy()
df_national_holiday['national_holiday_type'] = df_national_holiday['holiday_type']
df_national_holiday.drop(['locale', 'locale_name', 'holiday_type'], axis=1, inplace=True)
df_national_holiday.drop_duplicates(subset='date', keep="first", inplace=True)
df_national_holiday['date'] = pd.to_datetime(df_national_holiday['date'])
df_train = pd.merge(df_train, df_national_holiday, how='left', on=['date'])

In [ ]:
state_holidays = df_holiday[df_holiday['locale'] == 'Regional'].copy()
state_holidays['state'] = state_holidays['locale_name']
state_holidays['state_holiday_type'] = state_holidays['holiday_type']
state_holidays.drop(['locale', 'locale_name', 'holiday_type'], axis=1, inplace=True)
state_holidays['date'] = pd.to_datetime(state_holidays['date'])
df_train = pd.merge(df_train, state_holidays, how='left', on=['date', 'state'])

In [ ]:
city_holidays = df_holiday[df_holiday['locale'] == 'Local'].copy()
city_holidays['city'] = city_holidays['locale_name']
city_holidays['city_holiday_type'] = city_holidays['holiday_type']
city_holidays.drop(['locale', 'locale_name', 'holiday_type'], axis=1, inplace=True)
city_holidays.drop([265], axis=0, inplace=True)
city_holidays['date'] = pd.to_datetime(city_holidays['date'])
df_train = pd.merge(df_train, city_holidays, how='left', on=['date', 'city'])

In [ ]:
df_train

In [ ]:
df_train['holiday_type'] = np.nan
df_train['holiday_type'] = df_train['holiday_type'].fillna(df_train['national_holiday_type'])
df_train['holiday_type'] = df_train['holiday_type'].fillna(df_train['state_holiday_type'])
df_train['holiday_type'] = df_train['holiday_type'].fillna(df_train['city_holiday_type'])
df_train['holiday_type'] = df_train['holiday_type'].fillna('Not_holiday')
df_train.drop(['national_holiday_type','state_holiday_type','city_holiday_type'],axis=1,inplace= True)
df_train

In [ ]:
df_train.dtypes

In [ ]:
df_train['holiday_type'].unique()

In [ ]:
#df_train = pd.get_dummies(df_train,columns=['holiday_type','type'])
#df_train = pd.get_dummies(df_train,columns=['holiday_type','type','cluster'])
encoder = LabelEncoder()
df_train['city']= encoder.fit_transform(df_train['city'])
df_train['state']= encoder.fit_transform(df_train['state'])
df_train['type']= encoder.fit_transform(df_train['type'])
df_train['cluster']= encoder.fit_transform(df_train['cluster'])
df_train['holiday_type']= encoder.fit_transform(df_train['holiday_type'])

In [ ]:
df_train.drop(['day','holiday_type'],axis=1,inplace=True)
#df_train.drop(['Year'],axis=1,inplace=True)

In [ ]:
df_train.columns.values.tolist()

## Modeling 

In [ ]:
df_train['family'].unique()

In [ ]:
df_train = df_train.replace('BREAD/BAKERY','BREADBAKERY')

In [ ]:


## spilt family types 
all_family = ['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREADBAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD']
for family in all_family:

    temp = df_train.copy()
    temp = temp[temp['family'] == family]
    
    family_name = family
    
    if family == 'BREAD/BAKERY':
        family_name = 'BREADBAKERY'
    else:
        temp.to_csv('/kaggle/working/'+str(family_name)+'.csv', index=False)
    

visual the solution from output

In [ ]:
df_pet = pd.read_csv('/kaggle/working/PET SUPPLIES.csv')
df_pet.set_index('date',inplace=True)
df_pet.head(5)

In [ ]:
df_pet.info()

### Spilt Train and Testing the Dataset

In [ ]:
def spilt_train_data(df):
    
    df_use = df.copy()
    
    train_data = df_use[df_use.index <= '2017-07-31']
    test_data = df_use[df_use.index > '2017-07-31']
    
    X_train = train_data.loc[:, df_use.columns != 'sales']
    y_train = train_data['sales']
    
    X_test = test_data.loc[:, df_use.columns != 'sales']
    y_test = test_data['sales']
    
    return X_train,y_train,X_test,y_test

In [ ]:
from sklearn.metrics import mean_squared_log_error

#### XGBoosting 

In [ ]:
## create a model

def process_xgboost(df):
    
    ## create the model
    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                               n_estimators=2000,
                               max_depth=5,
                               learning_rate=0.05)
    ## spilt the data 
    X_train,y_train,X_test,y_test = spilt_train_data(df)
    
    ## fit the model
    reg.fit(X_train,y_train)
    
    
    ## predict test
    y_pred = reg.predict(X_test)
    
    #if the value is negative convert to 0
    y_pred = list(y_pred)
    y_pred = [max(0, x) for x in y_pred]

    return y_pred , y_test, reg

In [ ]:
df_pet.drop(['family'],axis=1,inplace= True)

In [ ]:
df_pet

In [ ]:
y_pred_xgb, y_test_xgb, reg = process_xgboost(df_pet)

In [ ]:
def evaluation(y_pred,y_test):
    y_pred = list(y_pred)
    y_test = list(y_test)
    print(f"Xgboost Regressor Score\n\n r squared: {r2_score(y_test, y_pred)}\n\n RMLSE: {mean_squared_log_error(y_test, y_pred, squared=False)}")
    

In [ ]:
evaluation(y_pred_xgb,y_test_xgb)

In [ ]:
def plot_sales_total_predicted(testing,y_pred,title='Total Sales using Xgboost Regressor'):
    testing_with_pred = testing.copy()
    testing_with_pred["sales_pred"] = y_pred
    
    testing_with_pred = testing_with_pred.groupby("date").sum()

    plt.figure(figsize=(25,7))
    act = plt.plot(testing_with_pred.index, testing_with_pred["sales"], color="blue")
    pred = plt.plot(testing_with_pred.index, testing_with_pred["sales_pred"], color="red")
    
    plt.xlabel("\nDate", size=15)
    plt.ylabel("Sales", size=15)
    
    plt.xticks(size=12)
    plt.yticks(size=12)
    
    plt.title(title, size=17)
    plt.legend(["Actual","Predicted"])
    plt.box(False)

In [ ]:
testing = df_pet[df_pet.index > '2017-07-31'].copy()
plot_sales_total_predicted(testing.copy(),y_pred_xgb)

In [ ]:
plt.figure(figsize=(5000,100))
plot_tree(reg,num_trees=4)

In [ ]:
# feature importance 
fi = pd.DataFrame(data=reg.feature_importances_,
            index=reg.feature_names_in_,
            columns=['importance'])
fi.sort_values('importance').plot(kind='barh',title='Feature Importance')
plt.show()

In [ ]:
testing

#### Random Forest

In [ ]:
def process_random_forest(df):
    
    ## create the model
    reg = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 18)
    ## spilt the data 
    X_train,y_train,X_test,y_test = spilt_train_data(df)
    
    ## fit the model
    reg.fit(X_train,y_train)
    
    
    ## predict test
    y_pred = reg.predict(X_test)
    
    #if the value is negative convert to 0
    y_pred = list(y_pred)
    y_pred = [max(0, x) for x in y_pred]

    return y_pred , y_test , reg

In [ ]:
df_pet

In [ ]:
y_pred_rf, y_test_rf,reg = process_random_forest(df_pet)

In [ ]:
evaluation(y_pred_rf,y_test_rf)

In [ ]:
testing = df_pet[df_pet.index > '2017-07-31'].copy()
plot_sales_total_predicted(testing.copy(),y_pred_rf)

In [ ]:
# feature importance 
fi = pd.DataFrame(data=reg.feature_importances_,
            index=reg.feature_names_in_,
            columns=['importance'])
fi.sort_values('importance').plot(kind='barh',title='Feature Importance')
plt.show()

In [ ]:
family_test = ['GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES']


for family in family_test:

    family_name = family
            
    df_temp = pd.read_csv('/kaggle/working/'+str(family_name)+'.csv')
    df_temp.set_index('date',inplace=True)
    df_temp.drop(['family'],axis=1,inplace= True)
    y_pred_solution, y_test_solution, reg = process_xgboost(df_temp)
    
    testing = df_temp[df_temp.index > '2017-07-31'].copy()
    plot_sales_total_predicted(testing.copy(),y_pred_xgb)


In [ ]:
for family in family_test:

    family_name = family
            
    df_temp = pd.read_csv('/kaggle/working/'+str(family_name)+'.csv')
    df_temp.set_index('date',inplace=True)
    df_temp.drop(['family'],axis=1,inplace= True)
    y_pred_solution, y_test_solution, reg = process_random_forest(df_temp)
    
    testing = df_temp[df_temp.index > '2017-07-31'].copy()
    plot_sales_total_predicted(testing.copy(),y_pred_xgb)
